In [1]:
import pandas as pd
import numpy as np
import datetime as dt
pd.options.display.max_columns = None
pd.options.display.max_rows=None

In [ ]:
lend = pd.read_csv('data/LoanStats3a.csv', header=1, low_memory=False)

#### Understanding data structure, checking shape and data type

In [ ]:
print(lend.shape)
lend.info()

There are 144 columns, 115 of those are floats data type

We will look at float data type columns first. From browsing the columns, it looks like they are information on describing the loans itself. Bear in mind that the problem is to predict whether the applicants should receive the loans, the details about the loans (such as terms, interest rates, hardship_length, etc.) are unknown upfront. Those columns will most likely to be ignored in the machine learning analysis.

In [ ]:
lend.select_dtypes(include='float').head()

Reviewing the 29 object columns. Some of the columns actually have date time or percentage values, but panda cannot recognize them in the initial load. Those columns data will need further cleansing

In [ ]:
lend.select_dtypes(exclude='float').head()

#### Keep only columns that few or no missing data. 

We will use isnull and sum function to check missing for each columns. Note that we have a total of 42K row of data. The result below indicates that some columns almost have no data at all, and wouldn't be useful. 

In [ ]:
lend.isnull().sum()

By scanning the missing records range, I will make the decision to drop columns that have >40K missing records.

In [ ]:
lend_1 = lend.dropna(axis=1, how='all', thresh=40000)
lend_1.shape

It appears to be 3 missing row in the csv, we will drop those as well.

In [ ]:
lend_2 = lend_1.dropna(subset=['loan_amnt'])
lend_2.shape

For the float type columns, we will replace missing values are equivalent to 0

In [ ]:
# Suppress output since it will take forever for the display
lend_2.select_dtypes(include='float').fillna(0);

For the object columns, further data type conversion and filling missing values will be need. We will focus on issue_d, emp_length, earliest_cr_line, revol_util and title. Other columns are either unknown variables for machine learning or no data type conversions are needed.

In [ ]:
lend_2.select_dtypes(exclude='float').isnull().sum()

#### Loan issues date
- Convert into datetime value

In [ ]:
#use assign function to avoid security type warining
lend_2 = lend_2.assign(loan_start_d=pd.to_datetime(lend_2.issue_d, format='%b-%Y'))

#### Employment duration
- convert from text to numeric value. 
- clean up na value, assume <1 year if info is not provided (currently unemployment)

In [ ]:
lend_2['yr_emp'] = lend_2['emp_length'].str[0:2].replace('< ',0).astype('float')

#### Credit History
- Convert earliest_cr_line into from text to datetime values
- Replace missing values with zero
- Extract year and convert to categorical values to have higher level grouping for this features

In [ ]:
lend_2.earliest_cr_line = pd.to_datetime(lend_2.earliest_cr_line, format='%b-%Y')
lend_2['yr_credit']= (2011-lend_2.earliest_cr_line.dt.year).fillna(0)

#### Credit line % usage
- Convert from string to decimal place

In [ ]:
lend_2['revol_util_dec'] = lend_2['revol_util'].str.replace(r'%',r'0').astype('float')/100

#### Identify the category values of loan_status, which will serve as target for this analysis

In [ ]:
lend_2.loan_status.value_counts()

#### Convert text value feature into numerics for model training purposes. Create a dictionary to convert loan status into 0 (default) or 1 (Paid)

Per research through discussion forum, "Dose not meet the credit policy' are loans that had issued before Lending Club update credit policy. For the purpose of this study, we will continue to treat them as "Fully Paid" or "Charged Off', regardless of credit policy compliance.

In [ ]:
loan_dict = {'Fully Paid': 1, 'Charged Off': 0, 'Does not meet the credit policy. Status:Fully Paid': 1, 'Does not meet the credit policy. Status:Charged Off': 0 }

#### Create a target column by mapping loan status to dictionary,

In [ ]:
#use assign function to avoid security type warining
lend_2 = lend_2.assign(target=lend_2.loan_status.map(loan_dict))

#### Identify potential features columns. 

Next step is to determine what features may have positive or negative correlation with % of loans being repay against total, which be done through visualization.

Save cleaned data set into a csv file for quicker access.

In [ ]:
lend_2.to_csv('data/lending_clean.csv')